# Examples from the gtfs_functions library

`gtfs_functions` is a python library maintained by a few guys who (I think) work or worked at Remix. It is supposed to be a neat, open-source way to parse and analyze gtfs feeds.

[Link to GitHub](https://github.com/Bondify/gtfs_functions)

## Notes for using
* Ideally, set up a dedicated conda environment to avoid conflicts.
* If you do this, you'll need to install pip with `conda install pip` before running pip to install the `gtfs_functions` library


Local debugging? Look in ...~\conda\envs\gtfs-funcs\Lib\site-packages\gtfs_functions


In [2]:
# Load library and an example transit feed
from gtfs_functions import Feed, map_gdf


feed_zip = r"I:\Projects\Darren\PEP\GTFS\2020 - SACSIM Base Year\zipped GTFS files\SRTD_1218_2019.zip"

# time windows refers to break points in day for determining frequency.
# E.g., if tw=[0, 7, 9, 18], then you'll get freqs computed for 12am-7am, 7am-9am, and 9am-6pm periods
feed = Feed(feed_zip, time_windows=[0, 7, 9, 15, 18])



In [15]:
# Loading and viewing basic GTFS files
routes = feed.routes # load table of routes
stops = feed.stops # load table of stops as geodataframe
trips = feed.trips
shapes = feed.shapes # loaded as geodataframe
shapes.head(3) # is geodataframe

INFO:root:Reading "routes.txt".
INFO:root:Reading "stops.txt".
INFO:root:accessing trips
INFO:root:Reading "trips.txt".
INFO:root:Reading "trips.txt".
INFO:root:computing patterns
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stop_times.txt".
INFO:root:Reading "shapes.txt".


,shape_id,geometry
0,34853,"LINESTRING (-121.26713 38.68008, -121.26712 38..."
1,34854,"LINESTRING (-121.38310 38.64554, -121.38311 38..."
2,34855,"LINESTRING (-121.50965 38.66564, -121.50956 38..."


In [3]:
# get frequency at each stop for each time_window period
stop_freq = feed.stops_freq
print(stop_freq.window.drop_duplicates())
stop_freq.head()

INFO:root:get trips in stop_times
INFO:root:accessing trips
INFO:root:Reading "routes.txt".
INFO:root:Reading "trips.txt".
INFO:root:Reading "trips.txt".
INFO:root:computing patterns
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:Reading "stop_times.txt".
INFO:root:Reading "stop_times.txt".


0      9:00-15:00
1     15:00-18:00
12      0:00-7:00
14      7:00-9:00
Name: window, dtype: object


,stop_id,direction_id,window,ntrips,min_per_trip,stop_name,geometry
0,1000,0,9:00-15:00,1,360,SLP DR & 43RD AVE (SB),POINT (-121.51478 38.51570)
1,1000,1,15:00-18:00,6,30,SLP DR & 43RD AVE (SB),POINT (-121.51478 38.51570)
2,1000,1,9:00-15:00,1,360,SLP DR & 43RD AVE (SB),POINT (-121.51478 38.51570)
3,1001,0,9:00-15:00,1,360,SLP DR & 43RD AVE (SB),POINT (-121.51548 38.51339)
4,1001,1,15:00-18:00,6,30,SLP DR & 43RD AVE (SB),POINT (-121.51548 38.51339)


In [22]:
# get frequency for each line for each time_window period
# line_freq  = feed.lines_freq
# line_freq.head()

In [4]:
# get sheduled speeds
speeds = feed.avg_speeds
speeds.head(3)

INFO:root:Reading "shapes.txt".
INFO:root:segments_df: 5326, geometry: 5326
INFO:root:adding runtime
INFO:root:adding distance in meters
INFO:root:calculating speed in km/h
INFO:root:fixing outliers
INFO:root:aggregating speed by segment and window


,route_id,route_name,direction_id,stop_sequence,segment_name,window,speed_kmh,avg_route_speed_kmh,segment_max_speed_kmh,runtime_sec,start_stop_name,end_stop_name,segment_id,start_stop_id,end_stop_id,shape_id,distance_m,geometry
0,001,1 GREENBACK,0,1,ARCADIA DR & GREENBACK LN (SB) - GREENBACK LN ...,0:00-7:00,21.6,22.6,21.559643,0.0,ARCADIA DR & GREENBACK LN (SB),GREENBACK LN & ARCADIA DR (WB),1184 - 3301,1184,3301,34853,179.576762,"LINESTRING (-121.26713 38.67913, -121.26713 38..."
1,001,1 GREENBACK,0,2,GREENBACK LN & ARCADIA DR (WB) - GREENBACK LN ...,0:00-7:00,21.0,22.6,21.000000,60.0,GREENBACK LN & ARCADIA DR (WB),GREENBACK LN & SUNRISE BLVD (WB),3301 - 1187,3301,1187,34853,355.743948,"LINESTRING (-121.26837 38.67847, -121.26895 38..."
2,001,1 GREENBACK,0,3,GREENBACK LN & SUNRISE BLVD (WB) - GREENBACK L...,0:00-7:00,21.6,22.6,27.000000,0.0,GREENBACK LN & SUNRISE BLVD (WB),GREENBACK LN & MERLINDALE DR (WB),1187 - 1188,1187,1188,34853,442.199839,"LINESTRING (-121.27246 38.67841, -121.27276 38..."


In [6]:
# export speeds to SHP
shp_out = r'I:\Projects\Darren\PEP\PEP_GIS\SHP\from_GTFS\test_seg_speeds.shp'
speeds.to_file(shp_out)